# Seleccion de origen-destino

In [19]:
# cell 0 - Añadir tfm_app al sys.path
import sys
import os

# Obtener ruta absoluta del directorio que contiene el notebook
notebook_dir = os.path.dirname(os.getcwd())  # sube un nivel desde /notebook
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

In [20]:
# cell 1 - Imports
from IPython.display import display
import folium

from map_interaction.select_points import create_map, process_geojson, save_geojson_to_disk
from utils.config import SAVE_DIR


In [ ]:
# cell 2 - Mostrar mapa interactivo con área de estudio
m = create_map()
display(m)

In [4]:
# cell 3 - Cargar el GeoJSON exportado (si lo exportaste localmente)
# Si lo hiciste desde un botón que descarga el archivo, colócalo tú en SAVE_DIR y luego cárgalo así:

import json

selection_path = os.path.join(SAVE_DIR, "selected_points.geojson")
with open(selection_path, 'r', encoding='utf-8') as f:
    geojson = json.load(f)

In [5]:
# cell 4 - Procesar y validar los puntos seleccionados
start, end = process_geojson(geojson)
print("Punto de inicio:", start)
print("Punto de destino:", end)

Punto de inicio: (38.907596, 1.993112)
Punto de destino: (39.964275, 3.426313)


# Producción de rutas

## Validamos puntos seleccionados

In [23]:
from map_interaction.select_points import get_validated_start_end_points

u, v = get_validated_start_end_points()
print("Inicio:", u)
print("Destino:", v)


Inicio: (38.907596, 1.993112)
Destino: (39.964275, 3.426313)


## Ingrasamos origen-destino como nodos en el grafo

In [2]:
from pathfinding.search import load_gpickle_compatible, run_dijkstra
from pathfinding.plot_route import plot_route_on_graph
from map_interaction.select_points import process_geojson, load_selection_geojson

In [ ]:
# Cargar grafo
grafo_path = os.path.join("..", "data", "raw", "grafo", "weather_routing_graph.gpickle")
G = load_gpickle_compatible(grafo_path)

In [1]:
# Cell 0 - Configurar ruta de proyecto
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

# Cell 1 - Imports de módulos
import pandas as pd
import json
import networkx as nx
from shapely.geometry import shape
from shapely.ops import unary_union

from utils.config import SAVE_DIR
from map_interaction.select_points import get_validated_start_end_points
from graph.env_data import fetch_depth, evaluate_navigability, batch_evaluate_navigability
from graph.wind_data import fetch_hourly_wind, evaluate_point_wind, batch_evaluate_wind
from graph.nautical_constraints import (
    fetch_seamark_elements, classify_seamarks,
    mark_blocked_seamark, create_buoy_nodes,
    combine_mesh_and_buoys, compute_navigable_final
)
from utils.geo_utils import haversine, bearing
from graph.calculate_weights import PolarDiagram
from graph.augment_graph import augment_with_start_end



In [3]:
# Cell 2 - Validar puntos START y END
start, end = get_validated_start_end_points()
print("START:", start)
print("END:  ", end)

START: (38.907596, 1.993112)
END:   (39.964275, 3.426313)


In [4]:
# Define parámetros de ejemplo
boat_data = {"draft": 1.5, "w_max": 25.0}
start_date, end_date = "2025-06-06", "2025-06-06"

In [5]:
# Cell 3 - Probar datos de batimetría
depth_start = fetch_depth(*start[::-1])  # fetch_depth expects lat, lon
print(f"Profundidad en START: {depth_start} m")
nav_start = evaluate_navigability(*start, boat_data["draft"])
print("START navegable por profundidad:", nav_start["navigable"])

Profundidad en START: None m
START navegable por profundidad: True


In [6]:
# Cell 4 - Probar datos de viento
wind_data_start = fetch_hourly_wind(start[0], start[1], start_date, end_date)
print("Primeros registros de viento en START:")
if wind_data_start:
    print({k: v[:3] for k, v in wind_data_start.items()})

Primeros registros de viento en START:
{'time': ['2025-06-06T00:00', '2025-06-06T01:00', '2025-06-06T02:00'], 'wind_speed_10m': [3.2, 1.6, 0.9], 'wind_direction_10m': [169, 173, 63]}


In [7]:
# Cell 5 - Batch evaluaciones
bathy_results = batch_evaluate_navigability([start, end], boat_data["draft"])
print("Batch batimetría:", bathy_results)
wind_results = batch_evaluate_wind(bathy_results, boat_data, start_date, end_date)
print("Número de registros de viento procesados:", len(wind_results))

Batch batimetría: [{'latitude': 38.907596, 'longitude': 1.993112, 'depth_avg': -667.2, 'navigable': True}, {'latitude': 39.964275, 'longitude': 3.426313, 'depth_avg': -137.83704, 'navigable': True}]
Número de registros de viento procesados: 48


In [8]:
# Cell 6 - Probar restricciones náuticas
# Cargar área para seamarks
area_path = os.path.join(SAVE_DIR, "selection.geojson")
with open(area_path, 'r', encoding='utf-8') as f:
    area_gj = json.load(f)
poly = shape(area_gj["features"][0]["geometry"])
minx, miny, maxx, maxy = poly.bounds
elements = fetch_seamark_elements(miny, minx, maxy, maxx)
restricted, buoys = classify_seamarks(elements)
print("Zonas restringidas:", len(restricted))
print("Boyas de peligro:", len(buoys))

Zonas restringidas: 14
Boyas de peligro: 198


In [9]:
# Crear un DataFrame de nodos inicial con START y END
nodes_df = pd.DataFrame([{
    "latitude": start[0], "longitude": start[1],
    "depth_avg": depth_start, "wind_speed_10m": None,
    "wind_direction_10m": None, "navigable": nav_start["navigable"]
}, {
    "latitude": end[0], "longitude": end[1],
    "depth_avg": fetch_depth(*end[::-1]),
    "wind_speed_10m": None, "wind_direction_10m": None,
    "navigable": bool(evaluate_navigability(*end, boat_data["draft"])["navigable"])
}])
nodes_df['blocked_seamark'] = mark_blocked_seamark(nodes_df, restricted)
buoy_df = create_buoy_nodes(buoys)
combined = combine_mesh_and_buoys(nodes_df, buoy_df)
combined['navigable_final'] = compute_navigable_final(combined)
print("Nodos combinados:")
print(combined)

Nodos combinados:
      latitude  longitude depth_avg wind_speed_10m wind_direction_10m  \
0    38.907596   1.993112      None           None               None   
1    39.964275   3.426313      None           None               None   
2    39.953173   0.045324      None           None               None   
3    38.979830   1.588132      None           None               None   
4    39.531828   2.588177      None           None               None   
..         ...        ...       ...            ...                ...   
195  38.846944   0.143333      None           None               None   
196  38.840139   0.152500      None           None               None   
197  38.833889   0.160556      None           None               None   
198  38.802639   0.202083      None           None               None   
199  39.896667   2.724167      None           None               None   

     navigable  blocked_seamark  is_buoy  time  navigable_final  
0         True            False    Fals

In [10]:
# Cell 7 - Probar utilidades geográficas y polar
# Distancia y rumbo entre START y END
dist = haversine(start[1], start[0], end[1], end[0])
brg = bearing(start[1], start[0], end[1], end[0])
print(f"Distancia START→END: {dist:.2f} m, Rumbo: {brg:.1f}°")


Distancia START→END: 170157.06 m, Rumbo: 45.9°


In [11]:
# Cargar diagrama polar de ejemplo
polar_df = pd.read_csv(os.path.join("..", "data", "raw", "malla", "polar_diagram.csv"))
polar = PolarDiagram(polar_df)
speed = polar.get_speed(brg)
print(f"Velocidad teórica para TWA={brg:.1f}°: {speed:.2f} kn")

Velocidad teórica para TWA=45.9°: 0.25 kn


In [12]:
# Cell 8 - Probar augment_graph
# Cargar grafo original
gpath = os.path.join("..", "data", "raw", "grafo_navegable", "weather_routing_graph.gpickle")
G = nx.read_gpickle(gpath)
# Cargar zonas prohibidas
nz_path = os.path.join("..", "data", "raw", "malla", "non_navigable_zones.json")
with open(nz_path, 'r') as f:
    nz = json.load(f)
union_restr = unary_union([shape(feat['geometry']) for feat in nz['features']])
# Aumentar grafo
G_aug = augment_with_start_end(G, start, end, union_restr, polar)
print("Grafo aumentado:", G_aug.number_of_nodes(), "nodos,", G_aug.number_of_edges(), "aristas")

Grafo aumentado: 4997 nodos, 112750 aristas


## Busqueda de camino

In [13]:
# 3) Aumentar el grafo con START y END
#    (start, end vienen de get_validated_start_end_points())
G_aug = augment_with_start_end(
    G,
    start=start,
    end=end,
    union_restr=union_restr,
    polar=polar,
    max_neighbors=32,
    neighbor_radius=0.05,
    alpha_time=1.0,
    beta_comfort=0.1
)

In [14]:
print("Grado START:", G_aug.degree('START'))
print("Sus sucesores:", list(G_aug.successors('START')))
print("Grado END:", G_aug.degree('END'))
print("Sus predecesores:", list(G_aug.predecessors('END')))


Grado START: 4
Sus sucesores: [2397, 2278, 2277, 2396]
Grado END: 4
Sus predecesores: [4609, 4498, 4497, 4608]


In [15]:
# 4) Ejecutar Dijkstra
result = run_dijkstra(G_aug, 'START', 'END', weight='weight')
print(f"Dijkstra → coste = {result['cost']:.2f}, "
      f"saltos = {result['hops']}, tiempo = {result['time']:.3f}s")

Dijkstra → coste = 269.34, saltos = 24, tiempo = 0.116s


In [16]:
# Cell – Visualizar la ruta sobre el grafo
from pathfinding.plot_route import plot_route_on_graph

# Asegúrate de usar las etiquetas de nodo correctas:
# Si llamaste a augment_with_start_end con labels 'START' y 'END', usa:
path = result['path']         # lista de nodos e.g. ['START', 2397,  ... , 'END']

# Dibuja la ruta en el grafo aumentado
m = plot_route_on_graph(G_aug, path, route_color='orange', node_color='lightblue')
m
